In [ ]:
"""
__SPICE OBJECT NAIF-ID CODES__
-3 : MOM, MARS ORBITER MISSION
-5 : AKATSUKI,PLC,VCO,
0  : SSB, SOLAR SYSTEM BARYCENTER
399004 : USUDA
399932 : IDSN32
"""

In [ ]:
# GEOMETRIES DONE

import numpy as np 
import matplotlib.pyplot as plt
import spiceypy as spice
import pandas as pd
import warnings
import itertools
warnings.filterwarnings("ignore")

spice_env = ('./kernels/requirements.txt')
spice.furnsh(spice_env) 
spice.ktotal('ALL')
#-----------------------------------------------------------------------------------------------------------------------------
GM_Venus    = 3.2468e14
GM_Earth    = 3.986e14
GM_Sun      = 1.327e20
GM_Mars     = 4.2828e13
f_sc        = 2296368750.0 
c           = 299792458.0
AU          = 1.496e+11
solar_radii = 6.95700e+8
#-----------------------------------------------------------------------------------------------------------------------------
probe    = input("Enter the name/NAIF code of the probe): ")            #'probe'
target   = input("Enter the name/NAIF code for the object of study): ") #'SUN'
observer = input("Enter the name/NAIF code of the observing station): ")#'observer32'
# probe    = 'MOM'
# target   = 'SUN'
# observer = 'IDSN32'
#-----------------------------------------------------------------------------------------------------------------------------
def orthoprojection(ax,ay,az,bx,by,bz,cx,cy,cz):
    abx = bx-ax
    aby = by-ay
    abz = bz-az
    acx = cx-ax
    acy = cy-ay
    acz = cz-az
    t = (abx*acx + aby*acy + abz*acz)/(abx*abx + aby*aby + abz*abz)
    px = ax + t*abx
    py = ay + t*aby
    pz = az + t*abz
    return px,py,pz
#-----------------------------------------------------------------------------------------------------------------------------
# step    = 100
step    = int(input("Enter the length of the observation period in seconds: "))
# subdivisions_per_second = 4
subdivisions_per_second = int(input("Number of parts to divide each second into: "))  # Number of parts to divide each second into
delta_T = 1 / subdivisions_per_second  # Delta_T defines the division of a second into smaller parts
utc = ['Oct 14, 2021']
# utc = [input("Enter the date of interest in the format Mon DD, YYYY): ")]
et1 = spice.str2et(utc[0])

# Generating a time series with steps defined in seconds and subdivided by delta_T
T2 = [et1 + x * delta_T for x in range(0, step * subdivisions_per_second)]
_, lightime = spice.spkezr(observer,T2,'J2000','NONE',probe)
T1  = [m-n  for m,n in zip(T2,lightime)]
#-----------------------------------------------------------------------------------------------------------------------------
print(np.shape(T1))
print("T1 : ",T1)
print("\n")
print(np.shape(T1))
print("T2 : ",T2)
print("\n")
#-----------------------------------------------------------------------------------------------------------------------------
for (t1,t2) in zip(T1,T2):
    print("#--------------------------------------------------------------------------------------------------------------------")
    print("t1: ",t1,"t2:",t2)
    print("#--------------------------------------------------------------------------------------------------------------------")

    probe_state_T1_S , _= spice.spkezr(probe,t1,'J2000', 'NONE', target)
    print(f"Shapes for state for {probe} : SME",np.shape(probe_state_T1_S))
    probe_state_T1_S    = np.array(probe_state_T1_S)
    print("#--------------------------------------------------------------------------------------------------------------------")

    observer_state_T2_S, _ = spice.spkezr(observer,t2,'J2000', 'NONE', target)
    print(f"Shapes for state for {observer} : SME",np.shape(observer_state_T2_S))
    observer_state_T2_S    = np.array(observer_state_T2_S)
    print("#--------------------------------------------------------------------------------------------------------------------")
    v_observer = observer_state_T2_S[3:6]*1000
    v_probe  = probe_state_T1_S[3:6]*1000
    print("Shapes for velocities : ", np.shape(v_observer), np.shape(v_probe))
    # print("#--------------------------------------------------------------------------------------------------------------------")
    # print("v_probe : ",v_probe)
    # print("#----------------------------------------------------------")
    # print("v_observer :",v_observer)
    print("#--------------------------------------------------------------------------------------------------------------------")
    r_probe_S   = (probe_state_T1_S[0:3]*1000)
    print(f"Shapes for Position for {probe} : ", np.shape(r_probe_S))
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_observer_S   = (observer_state_T2_S[0:3]*1000)
    print(f"Shapes for Position for {observer} : ", np.shape(r_observer_S))
    print("#--------------------------------------------------------------------------------------------------------------------")

    offset_point = orthoprojection((probe_state_T1_S[0]*1000),(probe_state_T1_S[1]*1000),(probe_state_T1_S[2]*1000),(observer_state_T2_S[0]*1000),(observer_state_T2_S[1]*1000),(observer_state_T2_S[2]*1000),0,0,0)
    print("Shapes for Offset point : ",np.shape(offset_point))
    print("Coordinates for offset point : ",offset_point)

    x = offset_point[0]
    y=offset_point[1]
    z = offset_point[2]
    coords = [x,y,z]
    RRsun = np.sqrt((x**2)+(y**2)+(z**2))
    offset_R = RRsun/solar_radii
    offset_rrsun = offset_R
    print("target offset in target radii : ",offset_rrsun)
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_probe_target_offset_abs =np.sqrt((r_probe_S[0]-offset_point[0])**2 + (r_probe_S[1]-offset_point[1])**2 + (r_probe_S[2]-offset_point[2])**2)
    print(f"Distance between {probe} and the point of closest approach to the {target} in AU : ",r_probe_target_offset_abs/AU) 

    r_earth_target_offset_abs =np.sqrt((r_observer_S[0]-offset_point[0])**2 + (r_observer_S[1]-offset_point[1])**2 + (r_observer_S[2]-offset_point[2])**2)
    print(f"Distance between the {observer} and the point of closest approach to the {target} in AU : ",r_earth_target_offset_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_probe_S_N = np.linalg.norm(r_probe_S) #[np.linalg.norm(rS)for rS in r_probe_S]
    r_probe_S_abs   = np.array(r_probe_S_N)
    print(f"Shapes for Position for {probe} after normalizing : ", np.shape(r_probe_S_abs))
    print("#--------------------------------------------------------------------------------------------------------------------")
    print(f"r_{probe}_S : ",r_probe_S_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    r_observer_S_N = np.linalg.norm(r_observer_S)# [np.linalg.norm(rIS) for rIS in r_observer_S]
    r_observer_S_abs   = np.array(r_observer_S_N)
    print(f"Shapes for Position for {observer} after normalizing : ", np.shape(r_observer_S_abs))
    print("#--------------------------------------------------------------------------------------------------------------------")
    print(f"r_{observer}_S : ",r_observer_S_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    rME_abs =np.sqrt((r_probe_S[0]-r_observer_S[0])**2 + (r_probe_S[1]-r_observer_S[1])**2 + (r_probe_S[2]-r_observer_S[2])**2)
    print(f"{observer} {probe} Distance in m : ",rME_abs)
    print(f"{observer} {probe} Distance in AU : ",rME_abs/AU)
    print("#--------------------------------------------------------------------------------------------------------------------")

    Card = np.arccos(((r_probe_S_abs/AU)**2 + (r_observer_S_abs/AU)**2 - (rME_abs/AU)**2)/(2*(r_probe_S_abs/AU)*(r_observer_S_abs/AU)))
    Card = np.nan_to_num(Card)
    print("ESP Angle in Radians : ",Card)
    print("#--------------------------------------------------------------------------------------------------------------------")

    Boat = np.arccos(((r_observer_S_abs/AU)**2 - (r_probe_S_abs/AU)**2 + (rME_abs/AU)**2)/(2*(rME_abs/AU)*(r_observer_S_abs/AU)))
    Boat = np.nan_to_num(Boat)
    print("SEP Angle in Radians : ",Boat)
    print("#--------------------------------------------------------------------------------------------------------------------")
    print("GEOMETRIES DONE")
    print("#--------------------------------------------------------------------------------------------------------------------")

geometries = {f'T1_{probe}':T1,
                  f'T2_{observer}':T2,
                  f'r_{observer}_{target}_AU':r_observer_S_abs/AU,
                  f'r_{probe}_{target}_AU':r_probe_S_abs/AU,
                  f'offset_r{target}':offset_rrsun,
                  f'{probe}_to_pt_of_closest_approach_dist_AU':r_probe_target_offset_abs/AU,
                  f'{observer}_to_pt_of_closest_approach_dist_AU':r_earth_target_offset_abs/AU,
                  f'{observer}_{probe}_dist_AU':rME_abs/AU,
                  f'{observer}_{target}_{probe}_angle':Card,
                  f'{target}_{observer}_{probe}_angle':Boat,
                  }

df = pd.DataFrame(geometries)
df.to_csv('geometries_test.txt',sep='\t',index=False)
print("Done.")